# Conversion of TXT to MAT for gm/ID result files for SG13G2

**(c) 2024 Boris Murmann and Harald Pretl**

Licensed under the Apache License, Version 2.0 (the “License”); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

In [1]:
import pandas as pd
import numpy as np
from numpy._core.records import fromarrays
from scipy.io import savemat

In [2]:
choice = 1  #start from 0
devices = ['sg13_lv_nmos', 'sg13_lv_pmos']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5])
nfing = np.array([1, 1])

In [3]:
df_raw = pd.read_csv('techsweep_'+devices[choice]+'.txt', sep=r'\s+')
par_names = df_raw.columns.to_list()
par_prefix = par_names[1].split('[')[0]

In [4]:
# remove extra headers in file body and unwanted columns
df_raw = df_raw[~df_raw['v-sweep'].str.contains('v-sweep')]
df = df_raw.drop(['v-sweep', 'v-sweep.1'], axis=1)
df = df.apply(pd.to_numeric)

# rename columns for readability
df.columns = df.columns.str.removeprefix(par_prefix+'[')
df.columns = df.columns.str.removesuffix(']')
df

119072 rows × 22 columns

In [5]:
# sweep variable vectors
l = (np.unique(abs(df['l']))*1e6).round(3)
vgs = np.unique(abs(df['g']))
vds = np.unique(abs(df['d']))
vsb = np.unique(abs(df['b']))

In [6]:
# data
# ngspice sweep order is vgs, vds, vsb, l
dims = [len(vgs), len(vds), len(vsb), len(l)]
id = np.reshape(df['ids'].values, dims, order='F')
vt = np.reshape(df['vth'].values, dims, order='F')
gm = np.reshape(df['gm'].values, dims, order='F')
gmb = np.reshape(df['gmb'].values, dims, order='F')
gds = np.reshape(df['gds'].values, dims, order='F')
cgsol = np.reshape(df['cgsol'].values, dims, order='F')
cgg = np.reshape(df['cgg'].values, dims, order='F') \
      + np.reshape(df['cgdol'].values, dims, order='F') \
      + np.reshape(df['cgsol'].values, dims, order='F')
cgb = -np.reshape(df['cgb'].values, dims, order='F')
cgd = -np.reshape(df['cgd'].values, dims, order='F') \
      + np.reshape(df['cgdol'].values, dims, order='F')
cgs = -np.reshape(df['cgs'].values, dims, order='F') \
      + np.reshape(df['cgsol'].values, dims, order='F')
cdd = np.reshape(df['cdd'].values, dims, order='F') \
      + np.reshape(df['cjd'].values, dims, order='F') \
      + np.reshape(df['cgdol'].values, dims, order='F')
css = np.reshape(df['css'].values, dims, order='F') \
      + np.reshape(df['cjs'].values, dims, order='F') \
      + np.reshape(df['cgsol'].values, dims, order='F')

# output order for standard mat file is l, vgs, vds, vsb
id = np.transpose(id, (3, 0, 1, 2))
vt = np.transpose(vt, (3, 0, 1, 2))
gm = np.transpose(gm, (3, 0, 1, 2))
gmb = np.transpose(gmb, (3, 0, 1, 2))
gds = np.transpose(gds, (3, 0, 1, 2))
cgg = np.transpose(cgg, (3, 0, 1, 2))
cgb = np.transpose(cgb, (3, 0, 1, 2))
cgd = np.transpose(cgd, (3, 0, 1, 2))
cgs = np.transpose(cgs, (3, 0, 1, 2))
cdd = np.transpose(cdd, (3, 0, 1, 2))
css = np.transpose(css, (3, 0, 1, 2))

In [7]:
dic = {
  "INFO": "IHP SG13G2, 130nm CMOS, PSP",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css
}

savemat(devices[choice]+'.mat', {devices[choice]: dic})